In [2]:
import time 
import numpy as np 
import matplotlib.pyplot as plt
from qcodes.instrument_drivers.american_magnetics.AMI430_visa import AMI430, AMI430_3D, AMI430SwitchHeater
from qcodes.math_utils.field_vector import FieldVector

In [3]:
iz = AMI430("z", address="TCPIP0::169.254.201.182::7180::SOCKET")
instrument = iz

Connected to: AMERICAN MAGNETICS INC. 430 (serial:430-22-055, firmware:3.15) in 1.45s


In [4]:
print('The use of a switchheater is enabeled', instrument.switch_heater.check_enabled())
print('switchheater current', instrument.switch_heater.current.get(),instrument.switch_heater.current.unit)
print('Heat time', instrument.switch_heater.heat_time.get(), instrument.switch_heater.heat_time.unit)
print('Cool time', instrument.switch_heater.cool_time.get(),instrument.switch_heater.cool_time.unit)


The use of a switchheater is enabeled True
switchheater current 20.1 mA
Heat time 30 s
Cool time 600 s


In [5]:

def set_calibration(instrument):
    # Coil settings
    # Order of initialitaion matter when the units were set to something strange. 
    instrument.field_units.set('tesla')
    instrument.ramp_rate_units('seconds')
    instrument.coil_constant.set(6/61.98) # Tesla/Ampere
    instrument.current_limit.set(61.98) # Ampere
    instrument.field_limit.set(6) # Tesla
    instrument.current_ramp_limit.set(0.0909) # Ampere/second
    instrument.field_ramp_limit.set(0.0909*6/61.98) # Tesla/second
    instrument.inductance.set(11)
    instrument.stability_parameter.set(0)
    instrument.ramp_rate.set(0.001) # Tesla/Second
    
    # Switch heater settings
    instrument.switch_heater.enable()
    instrument.switch_heater.heat_time.set(30) # seconds
    instrument.switch_heater.cool_time.set(600) # seconds
    instrument.switch_heater.current.set(20.1) # mA
    instrument.switch_heater.persistent_current_ramp_rate.set(10) #Ampere/s
    return 

set_calibration(instrument)



# Testing a full ramp with switch heater

- Option 1:
    - Read current on supply and magnet
    - Turn on switch heater 
    - Ramp to field 
    - Read the current on supply and magnet
    - Turn off switch heater 
    - Read the current on supply and magnet 
    - Turn on switch heater 
    - Read the current on supply and magnet 
    - Ramp down and turn off switch heater 
- Option 2: 
    - Same as Option 1 but check if we can deenergize the leads when the field is ramped.

## Option 1:

In [7]:
def print_currents(isntrument):
    print('Supply current', instrument.current_supply.get(), instrument.current_supply.unit)
    print('Magnet current', instrument.current_magnet.get(), instrument.current_magnet.unit)
    return 

In [11]:




#instrument.switch_heater.on()

#instrument.field.set(0.001)

In [12]:
#instrument.field.set(0.001)

In [13]:
#instrument.switch_heater.off()

In [14]:
#instrument.switch_heater.on()
#instrument.field.set(0)

It seems to me that there is no automated switch heater sweep command in the AMI 430 qcodes driver. the sates of the heaters have to be set manually.

I dont understand the difference between the two commands. 

instrument.switch_heater.state :
The switchheater state has False or True as a output 
Asks for PS?
instrument.switch_heater.in_persistent_mode
Returns true and false 
Asks for PERS?



From p177
• PSwitch {0|1}
Turns the persistent switch heater OFF and ON. Sending “0” turns the
switch heater OFF. Sending a “1” turns the switch heater ON. Them
default value is “0”.
• PSwitch?
Returns a “0” indicating the switch heater is OFF, or a “1” indicating the
persistent switch heater is ON.

• PERSistent?
Returns the state of the MAGNET IN PERSISTENT MODE LED on the
front panel of the Model 430: “0” if the LED is OFF; “1” if the LED is ON.




I dont understand exactly what the difference is here. 


From p 135 136 

We have the following commands available: 

-   CONFigure:PSwitch {0|1}
    0 No switch installed 1 switch installed 
-   CONFigure:PSwitch:CURRent <current (A)>
    Documentation on page 161 states that we set the state in mA makes more sense to me.
-   CONFigure:PSwitch:TRANsition {0|1}
    0 Timer based transition 1 Voltage based transition 

-   CONFigure:PSwitch:HeatTIME <time (sec)>
-   CONFigure:PSwitch:CoolTIME <time (sec)>
-   CONFigure:PSwitch:PowerSupplyRampRate <rate (A/s)>
    Ramp rate of the current once the persisten switch is in the cooled state.
    Can be a lot higher than the ramp rate we usually use. See p58 of the manual for further information.
-   CONFigure:PSwitch:CoolingGAIN <percent>
    Dont understand this.
-   PSwitch:INSTalled?
-   PSwitch:AUTODetect?
-   PSwitch:CURRent?
-   PSwitch:TRANsition?
-   PSwitch:HeatTIME?
-   PSwitch:CoolTIME?
-   PSwitch:PowerSupplyRampRate?
-   PSwitch:CoolingGAIN?


We follow the following ramp up procedure:

1.  CONFigure:PSwitch {0|1} set to 1 to indicate persistent switch is installed 
2.  CONFigure:PSwitch:CURRent <current (A)> then we set the current 
    -   The PSwitch:AUTODetect? command does an autodetect of the needed current. 
        We dont need this mehtod 
3.  CONFigure:PSwitch:HeatTIME <time (sec)>
4.  CONFigure:PSwitch:CoolTIME <time (sec)>
5.  CONFigure:PSwitch:PowerSupplyRampRate <rate (A/s)>


How does the Code perform a ramp with a persisitent switch installed. 
I focus only on the switch heater checks here. 

-   Check the state of the switch heater. Raise error if its not resisitive

I need to understand what the difference between switch heater enabled and switch heater on is.